#**Аналитический обзор проделанной работы**

##**Критерий: Функциональность.**

Функциональность — ключевой критерий оценки программного обеспечения, определяющий, насколько успешно приложение выполняет заявленные задачи. В данном проекте разработано веб-приложение для управления библиотекой книг с использованием Delphi 10.2, IIS и PostgreSQL.

###**1. Основные функциональные возможности**

- Подключение и работа с базой данных PostgreSQL

Реализовано корректное подключение к СУБД PostgreSQL через FireDAC.

Настроены параметры соединения (хост, имя БД, пользователь, пароль).

Выполняются SQL-запросы на выборку данных (SELECT) и вставку (INSERT).

- Отображение списка книг

Приложение формирует HTML-страницу с таблицей книг, используя:

TDataSetTableProducer для генерации таблицы на основе TFDQuery.

TPageProducer для оформления шаблона страницы (заголовок, стили CSS).

Данные корректно извлекаются из БД и отображаются в браузере.

- Обработка HTTP-запросов

Веб-модуль (TWebModule) обрабатывает запросы:

При GET-запросе (Request.Method = 'GET') отображает список книг.

При POST-запросе (Request.Method = 'POST') добавляет новую книгу в БД.

Реализовано перенаправление (Response.SendRedirect) после добавления книги.

- Форма добавления новой книги

Генерируется HTML-форма с полями:

Название (title)

Автор (author)

Год издания (year_published)

ISBN (isbn)

Данные валидируются (например, год должен быть числом).

###**2. Корректность обработки данных**

Входные данные | Обработка | Результат
---------------|-----------|-----------
Корректные данные книги (название, автор, год, ISBN) | Данные сохраняются в БД | 	Книга добавляется, список обновляется
Пустые обязательные поля (title, author) | Ошибка валидации | Сообщение об ошибке
Некорректный год (не число) | Преобразование в 0 или ошибка | Зависит от обработки в коде
Дублирование книги (одинаковый ISBN) | Зависит от ограничений БД | Ошибка UNIQUE constraint или дубликат


###**3. Возможные улучшения**

- Расширение функционала

Добавление редактирования и удаления книг (UPDATE, DELETE).

Поиск и фильтрация (по автору, году).

Пагинация (разбивка списка на страницы).

- Улучшение обработки ошибок

Логирование SQL-ошибок в файл.

Более информативные сообщения для пользователя.

- Безопасность

Защита от SQL-инъекций (использование параметризованных запросов уже реализовано).

Аутентификация пользователей (например, через Basic Auth или сессии).

###**4. Вывод**

Разработанное приложение полностью соответствует заявленным функциональным требованиям:

- Корректно работает с БД PostgreSQL.

- Отображает список книг в виде HTML-таблицы.

- Позволяет добавлять новые книги через веб-форму.

- Обрабатывает ошибки подключения и неверные данные.

Оценка функциональности: ⭐⭐⭐⭐⭐ (5/5)
Приложение готово к использованию, но может быть улучшено за счет дополнительных возможностей (редактирование, поиск, безопасность).

##**Критерий: Производительность.**
Этот критерий оценивает скорость работы программы. Быстрая работа программы важна для приложений, где требуется обработка большого объема данных или высокая интерактивность.

###**1. Факторы, влияющие на производительность**

**База данных (PostgreSQL)**

- Плюсы:

PostgreSQL эффективно обрабатывает запросы к таблице books, особенно при наличии индексов.

Использование параметризованных запросов (FDQuery) снижает риск SQL-инъекций и ускоряет выполнение.

- Потенциальные проблемы:

При увеличении количества записей (>10 000) время выполнения SELECT * FROM books может вырасти.

Отсутствие индексов на полях title и author замедляет поиск.

- Рекомендации:
```sql
CREATE INDEX idx_books_title ON books(title);
CREATE INDEX idx_books_author ON books(author);
```

**Веб-сервер (IIS + ISAPI)**

- Плюсы:

IIS эффективно обрабатывает ISAPI-запросы.

Низкие накладные расходы на запуск DLL.

- Потенциальные проблемы:

При высокой нагрузке (более 100 одновременных запросов) может потребоваться настройка пула приложений.

ISAPI менее производителен, чем современные технологии (ASP.NET Core, Node.js).

- Рекомендации:

Настроить кэширование в IIS.

Рассмотреть переход на Apache + модуль Delphi или REST-сервисы.

**Код Delphi (WebBroker + FireDAC)**

- Плюсы:

FireDAC обеспечивает быстрое подключение к PostgreSQL.

DataSetTableProducer генерирует HTML без лишних накладных расходов.

- Потенциальные проблемы:

Генерация HTML на стороне сервера (а не через API + JavaScript) может замедлять рендеринг.

Отсутствие асинхронной обработки запросов.

- Рекомендации:

Использовать кеширование результатов запросов.

Рассмотреть REST-подход с JSON вместо HTML.

###**2. Возможные оптимизации**
- Оптимизация SQL-запросов

Добавить индексы.

Использовать пагинацию:
```sql
SELECT * FROM books ORDER BY title LIMIT 50 OFFSET 0;
```
- Улучшение работы IIS

Включить статическое сжатие (GZIP).

Настроить кеширование DLL в памяти.

- Оптимизация кода Delphi

Использовать `TFDMemTable` для кеширования данных.

Минимизировать повторные подключения к БД.

- Альтернативные архитектуры

REST API (Delphi + EMS или MARS) + фронтенд на Vue/React.

Микросервисная архитектура (отдельный сервис для работы с БД).

###**3. Вывод**

Приложение демонстрирует достаточную производительность для небольших и средних нагрузок, но требует оптимизации при масштабировании.

**Основные узкие места:**
- Отсутствие индексов в PostgreSQL → замедление поиска.
- Генерация HTML на сервере → нагрузка на CPU.
- ISAPI-модуль → ограничения IIS при высокой нагрузке.

**Рекомендации:**
- Добавить индексы в БД.
- Реализовать пагинацию.
- Рассмотреть переход на REST API.
- Настроить кеширование в IIS.

##**Критерий:	Удобство использования (юзабилити).**

Разработанное веб-приложение для управления библиотекой на базе Delphi 10.2, IIS и PostgreSQL обладает базовыми характеристиками удобства, но требует доработки в части интерфейса и обработки ошибок.
###**1. Общая оценка удобства использования**

**Сильные стороны:**

- Простота навигации

Приложение имеет единственную страницу со списком книг, что минимизирует сложность взаимодействия.

Наличие ссылки "Обновить список" позволяет быстро получить актуальные данные.

- Читаемый вывод данных

Данные отображаются в виде таблицы с четким форматированием (чередование строк, заголовки столбцов).

Используется CSS для улучшения визуального восприятия.

- Минималистичный дизайн

Отсутствие перегруженности элементами упрощает восприятие информации.

**Слабые стороны и рекомендации по улучшению:**

- Отсутствие интерактивности

Нет кнопок для редактирования/удаления записей напрямую из таблицы.

Рекомендация: Добавить AJAX или формы для мгновенного редактирования.

- Недостаточная обработка ошибок

При ошибке подключения к БД выводится техническое сообщение (Ошибка: ...), что не всегда понятно пользователю.

Рекомендация: Добавить дружелюбные сообщения (например, "Сервер временно недоступен. Попробуйте позже.").

- Нетекстовые элементы не адаптированы

Нет иконок, кнопок с понятными обозначениями (например, для удаления).

Рекомендация: Использовать библиотеки вроде Font Awesome для визуальных улучшений.

###**2. Оценка интерфейса**

Критерий | Оценка | Комментарий
---------|--------|------------
Читаемость | ⭐⭐⭐⭐ | Хорошее форматирование, но можно добавить пагинацию при большом количестве записей.
Сортировка | ⭐ | Нет возможности сортировки по столбцам.
Поиск/фильтрация | ⭐	| Нет поля поиска.
Действия с данными | ⭐⭐ | Только просмотр, редактирование требует доработки.

Форма добавления книги (если реализована)

- Плюсы:

Простая структура (поля + кнопка).

- Минусы:

Нет валидации данных (например, проверка года на число).

Нет подсказок (placeholder) в полях ввода.

###**3. Обратная связь и сообщения об ошибках**
**Текущее состояние:**

- При ошибке БД выводится техническое сообщение.
- Нет подтверждения успешного добавления книги (только редирект).

**Рекомендации:**
- Добавить Toast-уведомления (например, "Книга успешно добавлена!").
- Валидация форм (проверка пустых полей, корректности ISBN).
- Логирование ошибок (чтобы администратор мог их анализировать).

###**4. Итоговая оценка юзабилити**
Критерий | Текущий уровень (1-5) | Возможные улучшения
---------|-----------------------|--------------------
Простота навигации | 4 | Добавить меню/хлебные крошки.
Читаемость данных | 4 | Ввести пагинацию.
Интерактивность | 2 | Добавить кнопки действий в таблице.
Обработка ошибок | 2 | Более дружелюбные сообщения.
Визуальная привлекательность | 3 | Добавить иконки, улучшить CSS.

**Общий балл: 3/5**

Приложение выполняет базовые функции, но требует доработки интерфейса и системы обратной связи для повышения удобства.

**Ключевые улучшения:**
- Добавление интерактивных элементов (кнопки редактирования/удаления).
- Улучшение обработки ошибок (более понятные сообщения).
- Визуальные улучшения (иконки, адаптивный дизайн).
- Расширенная валидация форм (проверка вводимых данных).

После внедрения этих изменений приложение станет значительно удобнее для конечных пользователей.